<a href="https://colab.research.google.com/github/iita71737/test-Colab-pyproject/blob/main/Hello_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
import json
import sqlite3
import datetime

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()


# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks
             (city text, counts integer, createdAt datetime)''')
c.execute('''DELETE FROM masks''')
conn.commit()

# 发送请求获取数据
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

# 检查请求是否成功
if response.status_code == 200:
    # 将响应内容解析为 JSON 格式
    data = json.loads(response.text)

    # 初始化成人剩余口罩数量字典
    mask_count = {}

    # 遍历数据计算每个地区的成人剩余口罩数量
    for pharmacy in data['features']:
        properties = pharmacy['properties']
        county = properties['county']
        adult_mask_count = int(properties['mask_adult'])

        # 如果该地区还没有记录，初始化为0
        if county not in mask_count:
            mask_count[county] = 0

        # 累加该地区的成人剩余口罩数量
        mask_count[county] += adult_mask_count

    # 将结果从大到小排序，并转换为字典形式
    mask_count_sorted = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse=True))

    # 打印计算结果
    print(mask_count_sorted)

    # 新增药局数量数据到数据库
    for county, counts in mask_count_sorted.items():
        t = datetime.datetime.now()
        print(f"INSERT INTO pharmacies VALUES ('{county}', {counts}, '{t}')")
        c.execute(f"INSERT INTO pharmacies VALUES ('{county}', {counts}, '{t}')")
        conn.commit()

    # 新增剩余口罩数量数据到数据库
    for county, counts in mask_count.items():
        t = datetime.datetime.now()
        print(f"INSERT INTO masks VALUES ('{county}', {counts}, '{t}')")
        c.execute(f"INSERT INTO masks VALUES ('{county}', {counts}, '{t}')")
        conn.commit()

    # 查询并打印 pharmacies 数据
    c.execute("SELECT * FROM pharmacies")
    print("Pharmacies Data:")
    print(c.fetchall())

    # 查询并打印 masks 数据
    c.execute("SELECT * FROM masks")
    print("Masks Data:")
    print(c.fetchall())

    # 可以选择将结果保存或进行其他操作
else:
    print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")


{'新北市': 670270, '臺中市': 536710, '高雄市': 424170, '臺南市': 393900, '桃園市': 331940, '臺北市': 322490, '彰化縣': 219400, '雲林縣': 196990, '屏東縣': 184250, '苗栗縣': 131080, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '': 69210, '花蓮縣': 57330, '新竹縣': 51350, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}
INSERT INTO pharmacies VALUES ('新北市', 670270, '2024-08-11 08:48:07.196400')
INSERT INTO pharmacies VALUES ('臺中市', 536710, '2024-08-11 08:48:07.205280')
INSERT INTO pharmacies VALUES ('高雄市', 424170, '2024-08-11 08:48:07.212854')
INSERT INTO pharmacies VALUES ('臺南市', 393900, '2024-08-11 08:48:07.220463')
INSERT INTO pharmacies VALUES ('桃園市', 331940, '2024-08-11 08:48:07.228173')
INSERT INTO pharmacies VALUES ('臺北市', 322490, '2024-08-11 08:48:07.235344')
INSERT INTO pharmacies VALUES ('彰化縣', 219400, '2024-08-11 08:48:07.242458')
INSERT INTO pharmacies VALUES ('雲林縣', 196990, '2024-08-11 08:48:07.250028')
INSERT INTO pharmacies VALUES ('屏東縣', 184250, '2024-08-11 08

In [9]:
from google.colab import auth
PROJECT_ID = "d12d12d" # @param {type: "string"}
auth.authenticate_user(project_id=PROJECT_ID)